In [1]:
import pandas as pd
import seaborn as sns

CWD = '/home/ubuntu/gnn_interpretation/src/testing/Balabanov_test/'

In [2]:
df = pd.read_csv("results_adam.csv", index_col='index')
cols_params = df.columns[:-5].to_list()
cols_metrics = df.columns[-5:].to_list()
cols_params, cols_metrics
df

,dataset,layer,K,aggr,layers_number,epochs_number,training_time,train_accuracy,train_F1,test_accuracy,test_F1
index,,,,,,,,,,,
1,Cora,GCNConv,NaN,add,1,170,3.494345,0.898430,0.889903,0.848708,0.832291
2,Cora,GCNConv,NaN,add,1,170,3.424329,0.897507,0.886286,0.830258,0.809624
3,Cora,GCNConv,NaN,add,1,170,2.671566,0.895660,0.886488,0.850554,0.848509
4,Cora,GCNConv,NaN,add,1,170,2.450644,0.887350,0.877077,0.835793,0.833147
5,Cora,GCNConv,NaN,add,1,170,3.022890,0.894737,0.888463,0.846863,0.819319
...,...,...,...,...,...,...,...,...,...,...,...
2475,AIDS,SAGEConv,NaN,mean,4,1400,169.399007,0.998125,0.997067,0.992500,0.988226
2476,AIDS,SAGEConv,NaN,mean,4,1400,168.505873,0.998750,0.998024,0.967500,0.952061
2477,AIDS,SAGEConv,NaN,mean,4,1400,164.513409,0.995625,0.993107,0.982500,0.972784


In [3]:
df = df.fillna(-1)

In [4]:
t = df.groupby(by=cols_params)#df.groupby(by=cols_params)
# t.mean().reset_index().to_csv("/home/fedor/gnn_interpretation/src/testing/Balabanov_test/mean.csv")
mean = t.mean().reset_index()
mean.columns = cols_params + [c + '_mean' for c in cols_metrics]

var = t.var().reset_index()
var.columns = cols_params + [c + '_var' for c in cols_metrics]

mv = pd.merge(mean, var, how='inner', on=cols_params)
mv.to_csv(CWD + 'mean_var.csv', float_format='%.3f')
mv

,dataset,layer,K,aggr,layers_number,epochs_number,training_time_mean,train_accuracy_mean,train_F1_mean,test_accuracy_mean,test_F1_mean,training_time_var,train_accuracy_var,train_F1_var,test_accuracy_var,test_F1_var
0,AIDS,GATConv,1.0,mean,2,800,101.400739,0.930875,0.886738,0.933500,0.887475,1.201460,0.000018,0.000047,0.000074,0.000211
1,AIDS,GATConv,1.0,mean,3,600,94.145846,0.980625,0.968835,0.984500,0.975912,0.196505,0.000074,0.000207,0.000048,0.000113
2,AIDS,GATConv,1.0,mean,4,450,81.553242,0.982625,0.972515,0.982000,0.970486,3.876020,0.000028,0.000072,0.000086,0.000264
3,AIDS,GCNConv,-1.0,add,2,1400,157.124819,0.949625,0.916105,0.945000,0.913915,8.398112,0.000149,0.000378,0.000122,0.000257
4,AIDS,GCNConv,-1.0,add,3,2000,243.277059,0.990875,0.985735,0.990500,0.984095,25.645747,0.000004,0.000009,0.000004,0.000024
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,PubMed,SGConv,1.0,-1,2,250,76.910985,0.849033,0.843966,0.846400,0.841358,0.531287,0.000003,0.000003,0.000062,0.000074
251,PubMed,SGConv,1.0,-1,3,250,84.445570,0.863691,0.855708,0.857809,0.849855,1.791510,0.000003,0.000005,0.000008,0.000024
252,PubMed,SSGConv,1.0,-1,1,500,166.228989,0.787206,0.775857,0.783722,0.772547,48.603671,0.000067,0.000070,0.000054,0.000059
253,PubMed,SSGConv,1.0,-1,2,300,104.624854,0.857643,0.852979,0.856542,0.852944,3.987224,0.000007,0.000005,0.000022,0.000022


In [5]:
t_layers = list(mv.layer.unique())

In [10]:
field, is_percent = ("test_accuracy", True)
# field, is_percent = ("training_time", False)
dataset = "MUTAG"

filename = f"tables/{field}_{dataset}.txt"

l_nums_by_dataset = {"Cora": [1, 2, 3], "BZR": [2, 3, 4], "MUTAG": [2, 3, 4]}
l_nums = l_nums_by_dataset[dataset]

num_cols = len(l_nums) + 1

# 1 слой	&	2 слоя	&	3 слоя

t_header = ("\\begin{table}[!htbp]\n"
            "\\centering\n"
            f"\\begin{{tabular}}{{{'c' * num_cols}}}\n")
t_footer = ("\\end{tabular}\n"
           f"\\caption{{Сравнение методов. {field} {dataset}}}\n"
           f"\\label{{tab:{field}_{dataset}}}\n"
           "\\end{table}\n")

with open(filename, "w") as f:
    f.write(t_header)
    f.write("Метод\t&\t1 слой &\t2 слоя &\t3 слоя\\\\\n")

    for l in t_layers:
        data = mv.query(f"layer == '{l}' & dataset == '{dataset}' & K in [-1, 1]")
        if data.empty:
            continue

        f.write(f"{l}\t&\t")
        for n in l_nums:
            mean = data.query(f"layers_number == {n}")[field + "_mean"].values
            var = data.query(f"layers_number == {n}")[field + "_var"].values
            if len(mean) != 1:
                continue
            mean, var = mean[0], var[0] ** 0.5
            if is_percent:
                mean *= 100
                var *= 100
            
            f.write(f"{mean:.2f}±{var:.2f}\t")
            if n == l_nums[-1]:
                f.write("\t\\\\\n")
            else:
                f.write("&\t")

    f.write(t_footer)



In [8]:
field, is_percent = ("test_accuracy", True)
# field, is_percent = ("training_time", False)
dataset = "Cora"
layer = "SGConv"

filename = f"tables/{field}_{layer}_{dataset}.txt"

l_nums_by_dataset = {"Cora": [1, 2, 3], "BZR": [2, 3, 4], "MUTAG": [2, 3, 4]}
l_nums = l_nums_by_dataset[dataset]

num_cols = len(l_nums) + 1

t_header = ("\\begin{table}[!htbp]\n"
            "\\centering\n"
            f"\\begin{{tabular}}{{{'c' * num_cols}}}\n")
t_footer = ("\\end{tabular}\n"
           f"\\caption{{Сравнение методов. {field} {layer} {dataset}}}\n"
           f"\\label{{tab:{field}_{dataset}}}\n"
           "\\end{table}\n")

with open(filename, "w") as f:
    f.write(t_header)
    f.write("k\t&\t1 слой &\t2 слоя &\t3 слоя\\\\\n")

    for k in [1, 2, 3, 4]:
        data = mv.query(f"layer == '{layer}' & dataset == '{dataset}' & K == {k}")
        if data.empty:
            continue

        f.write(f"{k}\t&\t")
        for n in l_nums:
            mean = data.query(f"layers_number == {n}")[field + "_mean"].values
            var = data.query(f"layers_number == {n}")[field + "_var"].values
            if len(mean) != 1:
                continue
            mean, var = mean[0], var[0] ** 0.5
            if is_percent:
                mean *= 100
                var *= 100
            
            f.write(f"{mean:.2f}±{var:.2f}\t")
            if n == l_nums[-1]:
                f.write("\t\\\\\n")
            else:
                f.write("&\t")

    f.write(t_footer)
